<a href="https://colab.research.google.com/github/ykitaguchi77/ImageProcessing/blob/master/Bilateral_eye_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#顔の画像から目を検出して切り抜くスクリプト
・Haarcascade_eyeを使用<br>
・目が検出できないものはskipする<br>
・検出された目周囲の画像を縦横とも1.1倍の幅で切り抜く
・切り取った画像を250px*250pxにresizeする<br>
・対応表をCSV形式で出力（連番、左右、元フォルダ、元画像名）

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import csv
from random import randint
from time import sleep
import math

import glob
import random
from PIL import Image
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode

#Google colabをマウント

In [66]:
from google.colab import drive
drive.mount('/content/drive')

!date -R
!unzip -qq drive/My\ Drive/Deep_learning/gravcont_original.zip
!date -R
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fri, 14 Feb 2020 17:40:27 +0000
replace cont/1000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Fri, 14 Feb 2020 17:40:33 +0000
cont  drive  grav  sample_data


In [0]:
#元画像フォルダ
in_path = '/content/cont'

#保存先フォルダ
out_path = '/content/drive/My Drive/Deep_learning/cont_processed'

#CSVファイルのフォルダ
#csv_path = '/content/drive/My Drive/Deep_learning/eyelid_images/eyelid_images.csv'


#Haarcascadeを指定

In [0]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

#処理するdirectoryの設定
（画像が多い時には複数回に分ける）

In [84]:
#処理するDirectoryの設定
file = os.listdir(in_path)
print(len(file))

#ここにフォルダ番号を記載する (ex. [0:999])
processing_file = file[0:322]
print(processing_file)
len(processing_file)

322
['1348.jpg', '1165.jpg', '1531.jpg', '1434.JPG', '605.jpg', '813.jpg', '547.jpg', '858.JPG', '132.jpg', '623.jpg', '1405.JPG', '310.jpg', '1256.JPG', '1054.jpg', '349.jpg', '689.jpg', '1102.jpg', '490.JPG', '1170.jpg', '1494.jpg', '1101.JPG', '112.jpg', '1495.jpg', '311.JPG', '1059.jpg', '1019.jpg', '548.jpg', '324.JPG', '943.jpg', '934.jpg', '726.jpg', '1122.jpg', '611.jpg', '66.jpg', '1361.jpg', '1066.jpg', '225.jpg', '13.JPG', '1461.jpg', '1281.jpg', '1291.jpg', '1254.JPG', '116.JPG', '1542.jpg', '809.JPG', '401.jpg', '1249.jpg', '711.jpg', '110.jpg', '1459.jpg', '402.JPG', '1463.jpg', '38.JPG', '388.jpg', '347.jpg', '1547.jpg', '923.jpg', '372.JPG', '1372.jpg', '259.jpg', '1134.jpg', '1014.JPG', '976.jpg', '444.jpg', '1086.jpg', '1045.JPG', '398.jpg', '589.jpg', '775.jpg', '1462.JPG', '1468.JPG', '124.jpg', '210.jpg', '735.jpg', '641.jpg', '1369.jpg', '267.jpg', '948.JPG', '1303.JPG', '1319.jpg', '1428.jpg', '871.JPG', '179.JPG', '98.jpg', '513.jpg', '345.jpg', '1145.jpg', '134

322

#画像切り抜き
大きい辺を1辺とする正方形に整形する

In [0]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, (0, (height - width) // 2))
        return result

In [86]:
#処理時間の計測
start = time.time()

l=0
for i in processing_file:
      l+=1
      print('')
      print(str(l))
      print(str(i))
      print('image path = '+ in_path + '/' + i)

      #OPEN CVで画像を読み込み
      img = cv2.imread(in_path + '/' + i) 
      
      # 画像グレースケール化
      grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      #100pix以上のもので目に見えるものを抽出
      eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(100, 100))
                    
      # 眼検出判定
      if len(eye_list) == 2:
          print('目が' + str(len(eye_list)) +'個検出されました')
      else:
          print("eye detection error")
      
      #画像の切り抜きと保存
      if len(eye_list) == 2:              
          ex_min, ey_min, ex_ew_max, ey_eh_max = 100000, 100000, 0, 0
          m=0
          for (ex, ey, ew, eh) in eye_list:
              m+=1
              #右眼の外上端、左眼の外下端の座標を取得
              ex_min, ey_min, ex_ew_max, ey_eh_max = min(ex_min, ex), min(ey_min, ey), max(ex_ew_max, ex+ew), max(ey_eh_max, ey+eh)
              print(ex_min, ey_min, ex_ew_max, ey_eh_max)
          img = Image.open(in_path + '/' + i)

          #横幅に余裕を持つように切り取った上で、横辺250pixに縮小する
          #img_cropped = img.crop((int(ex_min), int(ey_min), int(ex_ew_max), int(ey_eh_max))) 
          img_cropped = img.crop((int(0.8*(ex_min)+0.2*(ex_ew_max)), int(ey_min), int(0.2*(ex_min)+0.8*(ex_ew_max)), int(ey_eh_max))) 
          img_new = expand2square(img_cropped, (0, 0, 0)).resize((250, 250))
          
          #切り取った画像を表示
          #plt.imshow(np.asarray(img_new))
          #plt.show()

          #ランダムな位置で224pixに切り取る
          #x, y = random.randint(0,27), random.randint(0,27) 
          #img_randcrop = img_resized[y: y+223, x: x+223]
          
          result.save(out_path +'/'+ str(i))
          print(out_path +'/'+ str(i))

print('Process done!!')
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")



1
1348.jpg
image path = /content/cont/1348.jpg
目が2個検出されました
203 434 922 1153
203 434 2171 1161
/content/drive/My Drive/Deep_learning/cont_processed/1348.jpg

2
1165.jpg
image path = /content/cont/1165.jpg
目が2個検出されました
170 585 804 1219
170 542 2252 1231
/content/drive/My Drive/Deep_learning/cont_processed/1165.jpg

3
1531.jpg
image path = /content/cont/1531.jpg
eye detection error

4
1434.JPG
image path = /content/cont/1434.JPG
目が2個検出されました
202 457 714 969
202 400 1790 1002
/content/drive/My Drive/Deep_learning/cont_processed/1434.JPG

5
605.jpg
image path = /content/cont/605.jpg
目が2個検出されました
1488 642 2056 1210
252 642 2056 1211
/content/drive/My Drive/Deep_learning/cont_processed/605.jpg

6
813.jpg
image path = /content/cont/813.jpg
eye detection error

7
547.jpg
image path = /content/cont/547.jpg
eye detection error

8
858.JPG
image path = /content/cont/858.JPG
eye detection error

9
132.jpg
image path = /content/cont/132.jpg
目が2個検出されました
2784 982 4178 2376
434 964 4178 2435
/content/driv

#切り取られた図を表示してみる

In [46]:
img_list = os.listdir(out_path)
print(img_list)
img = cv2.imread(out_path +'/'+ file[0])

#アスペクト比を維持したまま横を400pixelに縮小する
def scale_to_width(img, width):
    scale = width / img.shape[1]
    return cv2.resize(img, dsize=None, fx=scale, fy=scale)
dst = scale_to_width(img, 400)

#図を表示する
cv2_imshow(dst)

[]


AttributeError: ignored